# Store Sales: Data Visualization 

In [70]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [71]:
%run -n main.py

In [72]:
df = pd.read_csv("../datasets/train.csv", index_col="date", parse_dates=True)
print(df.index.dtype)
df.head()

datetime64[ns]


,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [73]:
weekly = df.drop(["store_nbr", "id"], axis=1).resample("W").sum()
weekly.head()

,sales,onpromotion
date,,
2013-01-06,2.211570e+06,0
2013-01-13,2.373618e+06,0
2013-01-20,2.368007e+06,0
2013-01-27,2.272165e+06,0
2013-02-03,2.476891e+06,0


In [74]:
group_sales = weekly.groupby(weekly.index)["sales"].sum()
monthly_rolling = group_sales.rolling(12, center=True)
yearly_rolling = group_sales.rolling(48, center=True)
data = pd.DataFrame({"date":pd.to_datetime(weekly.index),
                     "sales":weekly["sales"],
                     "monthly_rolling_mean":monthly_rolling.mean(),
                     "yearly_rolling_mean":yearly_rolling.mean()})

fig = px.line(data, x="date", y=["sales", "monthly_rolling_mean", "yearly_rolling_mean"], title="Sales over time")
fig.update_traces(line=dict(width=3))
fig.add_vline(x="2016-04-16", line_dash="dash", line_width=3)
fig.add_annotation(x="2016-04-16", y=0.8, yref="paper",
            text="Earthquake", font_size=18)
fig.show()

In [88]:
weekly_type = df.drop(["family", "id", "store_nbr"], axis=1).groupby(df["store_nbr"]).resample("W").sum().reset_index()
fig = px.line(weekly_type, x="date", y="sales", color="store_nbr", title="Sales by Product")
fig.add_vline(x="2016-04-16", line_dash="dash", line_width=2)
fig.add_annotation(x="2016-04-16", y=0.8, yref="paper",
            text="Earthquake", font_size=18)
fig.show()

In [87]:
from scipy.stats import zscore
df["sales_zscore"] = df.groupby("family")["sales"].transform(lambda x : zscore(x,ddof=1))
norm_weekly_type = df.groupby("family")["sales_zscore"].resample("W").mean().reset_index()
fig = px.line(norm_weekly_type, x="date", y="sales_zscore", color="family", title="Sales Z-Scores by Product")
fig.add_vline(x="2016-04-16", line_dash="dash", line_width=2)
fig.add_annotation(x="2016-04-16", y=0.8, yref="paper",
            text="Earthquake", font_size=18)
fig.show()

In [ ]:
by_weekday = df.groupby(df.index.dayofweek).mean()
by_weekday.index = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
fig = px.bar(by_weekday, x=by_weekday.index, y="sales", title="Sales by Weekday")
fig.show()